In [1]:
import cv2
import torch
import torchvision
from torch2trt import TRTModule

device = torch.device('cuda')
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('OBSTACLE2_trt.pth'))

<All keys matched successfully>

In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [3]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.csi_camera import CSICamera
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpeg', value)[1])

camera = CSICamera(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

camera.running = True

display(widgets.VBox([widgets.HBox([image, blocked_slider])]))

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [5]:
import torch.nn.functional as F
import time

def drive_forward(car):

    car.throttle = 0
    car.steering = 0

def blocked(car):
    
    car.throttle = 0
    car.steering = 0

def update(change):
    global blocked_slider, car
    x = change['new'] 
    x = preprocess(x)
    a=time.time()*1000
    y = model_trt(x)
    print(f"temps d'inférence:{(time.time()*1000)-a}")

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        drive_forward(car)
    else:
        blocked(car)
        
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

temps d'inférence:1463.812744140625


In [6]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

temps d'inférence:30.037109375
temps d'inférence:1.87255859375
temps d'inférence:1.8115234375
temps d'inférence:1.774169921875
temps d'inférence:2.154541015625
temps d'inférence:1.98486328125
temps d'inférence:1.691650390625
temps d'inférence:1.669189453125
temps d'inférence:1.718017578125
temps d'inférence:1.642578125
temps d'inférence:1.525390625
temps d'inférence:1.580810546875
temps d'inférence:1.754150390625
temps d'inférence:1.88916015625
temps d'inférence:1.836669921875
temps d'inférence:1.610107421875
temps d'inférence:1.575927734375
temps d'inférence:1.723876953125
temps d'inférence:1.80859375
temps d'inférence:1.73388671875
temps d'inférence:1.630126953125
temps d'inférence:1.5517578125
temps d'inférence:1.54931640625
temps d'inférence:1.831298828125
temps d'inférence:1.904541015625
temps d'inférence:1.6669921875
temps d'inférence:1.821044921875
temps d'inférence:1.58642578125
temps d'inférence:1.743408203125
temps d'inférence:1.60986328125
temps d'inférence:1.6220703125
temp